In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from prettytable import PrettyTable
from datetime import datetime, timedelta
from tqdm import tqdm
import statsmodels.api as sm
import yfinance as yf
import numpy as np

In [2]:
ANOS_PASSADOS=5

In [3]:
def parse_bcb(codigo_serie, indice):
    # Obtendo a data atual
    hoje = datetime.now()
    
    # Calculando as datas de início e fim
    data_inicio = f"{hoje.day}/{hoje.month}/{hoje.year-ANOS_PASSADOS}"
    data_final = f"{hoje.day}/{hoje.month}/{hoje.year}"

    # Construindo a URL para a API
    URL = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=csv&dataInicial={data_inicio}&dataFinal={data_final}"
    #print(URL)
    # Lendo o CSV do link fornecido
    df = pd.read_csv(URL, sep=";")

    # Convertendo a coluna "data" para o formato datetime
    # Aqui, ajustei o formato da data para corresponder ao formato fornecido: %d/%m/%Y
    df.rename(columns={"data": "date"}, inplace=True)
    df["date"] = pd.to_datetime(df["date"], format='%d/%m/%Y')
    df.set_index('date', inplace=True)
    df["valor"] = df["valor"].str.replace(',', '.').astype(float)
    df.rename(columns={"valor": indice}, inplace=True)
    return df


In [38]:
selic_df=parse_bcb(11, "Selic")
ipca_df=parse_bcb(10844, "IPCA")
SELIC=(selic_df.mean().values[0]*22*12)/100
IPCA=(ipca_df.mean().values[0]*12)/100
ANO_DIAS_UTEIS=252

In [19]:
mercado=["^BVSP",]
acoes=["WEGE3", "MGLU3", "VALE3", "GGBR4"]
acoes=[t+ ".SA" for t in acoes ]
tickers=[t.upper() for t in acoes+mercado]
tickers

['WEGE3.SA', 'MGLU3.SA', 'VALE3.SA', 'GGBR4.SA', '^BVSP']

In [21]:
hoje = datetime.now()
start_date=f"{hoje.year-ANOS_PASSADOS}-{datetime.now().month}-{datetime.now().day}"
data= yf.download(tickers, start_date)['Adj Close']

[*********************100%%**********************]  5 of 5 completed


In [27]:
mercado_df=data[tickers[-1]]
acoes_df=data[tickers[:-1]]

In [28]:
retorno_diario = acoes_df.pct_change()
retorno_anual = retorno_diario.mean() * ANO_DIAS_UTEIS
cov_diaria = retorno_diario.cov()
cov_anual = cov_diaria * ANO_DIAS_UTEIS

In [42]:
# beta_alpha 360
def alpha_beta(data, ticker, indice):
   
    #Calcular os retornos logarítmicos
    retorno_ativos = np.log(data/data.shift(1))

    # Remover os NaNs
    retorno_ativos = retorno_ativos.dropna()

    # Definir y (ticker) e X (indice)
    y = retorno_ativos[ticker]
    X = retorno_ativos[indice]

    # Adicionar uma constante ao X (para o intercepto)
    X = sm.add_constant(X)

    # Realizar a regressão
    model = sm.OLS(y,X).fit()

    # Coletar o alpha e beta
    alpha = model.params[0]
    beta = model.params[1]
    
    return alpha,beta


a,b = alpha_beta(data,"MGLU3.SA","^BVSP")
a,b

(-0.0010438556519832515, 1.3658696823640757)

In [52]:
def retorno_esperado(beta,taxa_livre_risco, premio_risco):
    re = taxa_livre_risco + beta * (premio_risco-taxa_livre_risco)
    return  re
re = retorno_esperado(b,SELIC,IPCA)


In [53]:
def vol_anual(data,ticker):
    daily_returns = np.log(data/data.shift(1))
    daily_returns = daily_returns[ticker].dropna()
    return daily_returns.std() * (252**0.5)

vol_anual(acoes_df, "MGLU3.SA")

0.6455170831868039

In [54]:
def sharpe_ratio(data, ticker, retorno_esperado, taxa_livre_risco):
    
    std = vol_anual(data, ticker) 
    # Calcula o Sharpe Ratio
    sharpe = (retorno_esperado - taxa_livre_risco) / std
    
    return sharpe
sharpe_ratio(acoes_df, "MGLU3.SA", re, SELIC)


-0.06353163796658058

In [55]:
def vol_anual_ajustada(data,ticker):
    daily_returns = np.log(data/data.shift(1))
    daily_returns = daily_returns[ticker].dropna()
    downside = daily_returns[daily_returns < 0]
    return downside.std() * (252**0.5)

vol_anual_ajustada(acoes_df, "MGLU3.SA")

0.4736808789565618

In [56]:
def sortino_ratio(data, ticker, retorno_esperado, taxa_livre_risco):
    std = vol_anual_ajustada(data,ticker)
    
    # Calcula o Sharpe Ratio
    sortino = (retorno_esperado - taxa_livre_risco) / std
    
    return sortino
sortino_ratio(acoes_df, "MGLU3.SA", re, SELIC)

-0.08657887504474913

In [ ]:
retorno_carteira = []
peso_acoes = []
volatilidade_carteira = []
sharpe_ratio = []
volatilidade_carteira_ajustada = []
sortino_ratio = []

In [57]:
# vamos usar uma simulação aleatória
numero_acoes = len(ativos)
numero_carteiras = 100000

np.random.seed(101)

In [ ]:
# aqui que preciso entender melhor esse montecarlo


# vamos fazer um for loop para preencher as lista que criamos anteriormente
for cada_carteira in range(numero_carteiras):
    # vamos dar um peso aleatório para cada ação dentro de cada carteira
    peso = np.random.random(numero_acoes)
    peso /= np.sum(peso)
    # vamos calcular o retorno das carteiras
    retorno = np.dot(peso, retorno_anual)
    # vamos calcular a volatilidade das carteiras
    volatilidade = np.sqrt(np.dot(peso.T, np.dot(cov_anual, peso)))
    # vamos calcular o índice de Sharpe de cada carteira
    sharpe = retorno / volatilidade
    # aqui nós usamos o método apend para incluir cada carteira nas listas criadas anteriormente
    sharpe_ratio.append(sharpe)
    retorno_carteira.append(retorno)
    volatilidade_carteira.append(volatilidade)
    peso_acoes.append(peso)
volatilidade